In [7]:
from pyspark.sql import SparkSession
from config import *

import os
# os.putenv("AWS_REGION","us-east-1") # this does not work
os.environ["AWS_REGION"] = "us-east-1"
import logging

# Set environment variable for log4j configuration file
os.environ['SPARK_CONF_DIR'] = 'log4j'

# Ensure log4j configuration is loaded
os.environ['SPARK_SUBMIT_OPTS'] = "spark.driver.extraJavaOptions=-Dlog4j.configuration=file://log4j.properties"

# logging.basicConfig(level=logging.ERROR, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")
# logger = logging.getLogger("Connectwise Lakehouse")

# logger.warning("starting")

In [8]:
spark = (
 SparkSession.builder
 .master("local")
 .config("spark.jars.packages",
      "org.apache.spark:spark-sql_2.12:3.5.1,"
      "org.apache.hadoop:hadoop-aws:3.3.4,"
      "org.apache.hadoop:hadoop-client-runtime:3.3.4,"
      "software.amazon.awssdk:bundle:2.23.19,"
      "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2,")
 #    // Iceberg Catalog Configuration
 .config("spark.sql.extensions",
      "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")  # Use Iceberg with Spark
 .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog")
 .config(f"spark.sql.catalog.{catalog_name}.type", "rest")
 .config(f"spark.sql.catalog.{catalog_name}.uri", f"http://{rest_endpoint}/")
 .config(f"spark.sql.catalog.{catalog_name}.warehouse", f"s3a://{bucket}")
 #  MinIO-Specific Configurations (for both Iceberg and Hadoop)
 .config(f"spark.sql.catalog.{catalog_name}.hadoop.fs.s3a.endpoint", f"http://{minio_endpoint}")
 .config(f"spark.sql.catalog.{catalog_name}.s3.endpoint", f"http://{minio_endpoint}")  # Added for Iceberg
 .config(f"spark.hadoop.fs.s3a.endpoint", f"http://{minio_endpoint}")
 # MinIO Credentials
 .config(f"spark.sql.catalog.{catalog_name}.hadoop.fs.s3a.access.key", access_key)
 .config(f"spark.sql.catalog.{catalog_name}.hadoop.fs.s3a.secret.key", secret_key)
 .config(f"spark.sql.catalog.{catalog_name}.s3.access-key-id", access_key)
 .config(f"spark.sql.catalog.{catalog_name}.s3.secret-access-key", secret_key)
 .config("spark.hadoop.fs.s3a.access.key", access_key)
 .config("spark.hadoop.fs.s3a.secret.key", secret_key)
 # Additional S3 Configurations
 .config("spark.hadoop.fs.s3a.path.style.access", "true")
 .config("spark.hadoop.fs.s3a.connection.establish.timeout", "15000")
 .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

 # Ensure Iceberg uses the correct S3 FileIO implementat
 # ion

 .getOrCreate()
)

Error: Could not find or load main class spark.driver.extraJavaOptions=-Dlog4j.configuration=file:..log4j.properties
Caused by: java.lang.ClassNotFoundException: spark/driver/extraJavaOptions=-Dlog4j/configuration=file://log4j/properties


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [56]:
print(spark.version)
spark.sql(f"USE {catalog_name}")

3.5.1
18:09:53.627 [Thread-2] DEBUG org.apache.spark.sql.execution.SparkSqlParser -- Parsing command: USE rest_catalog


DataFrame[]

In [57]:
spark.sparkContext.setLogLevel('ERROR')

join_query = \
 f'''
 SELECT *
 FROM {namespace}.customers 
--    LEFT JOIN {namespace}.devices ON {namespace}.customers.customer_id = {namespace}.devices.customer_id
--    LEFT JOIN {namespace}.device_statistics ON {namespace}.devices.device_id = {namespace}.device_statistics.device_id
--    GROUP BY  {namespace}.customers.customer_id
 '''

joined = spark.sql(join_query)

18:09:55.548 [Thread-2] DEBUG org.apache.spark.sql.execution.SparkSqlParser -- Parsing command: 
 SELECT *
 FROM ncentral.customers 
--    LEFT JOIN ncentral.devices ON ncentral.customers.customer_id = ncentral.devices.customer_id
--    LEFT JOIN ncentral.device_statistics ON ncentral.devices.device_id = ncentral.device_statistics.device_id
--    GROUP BY  ncentral.customers.customer_id
 
18:09:55.555 [Thread-2] DEBUG org.apache.iceberg.CachingCatalog -- Evicted ncentral.customers from the table cache (EXPIRED)
18:09:55.557 [Thread-2] DEBUG org.apache.iceberg.shaded.org.apache.hc.client5.http.impl.classic.InternalHttpClient -- ex-0000000042 preparing request execution
18:09:55.558 [Thread-2] DEBUG org.apache.iceberg.shaded.org.apache.hc.client5.http.impl.classic.ProtocolExec -- ex-0000000042 target auth state: UNCHALLENGED
18:09:55.558 [Thread-2] DEBUG org.apache.iceberg.shaded.org.apache.hc.client5.http.impl.classic.ProtocolExec -- ex-0000000042 proxy auth state: UNCHALLENGED
18:09:55

In [64]:

import logging

# Suppress all logging
logging.getLogger().setLevel(logging.ERROR)

# If you want to be specific, you can configure the py4j logger as well
logging.getLogger('py4j').setLevel(logging.ERROR)
spark.sparkContext.setLogLevel('ERROR')
print("customers_devices-count: ", joined.count())

18:14:16.920 [Thread-2] INFO org.apache.iceberg.SnapshotScan -- Scanning table rest_catalog.ncentral.customers snapshot 8961327026375986033 created at 2024-06-26T14:54:58.521+00:00 with filter true
18:14:16.925 [Thread-2] DEBUG org.apache.iceberg.aws.s3.S3InputStream -- Seek with new stream for s3a://lakehouse/ncentral/customers/metadata/a65fa511-8871-4ae5-9825-803acc4863ef-m0.avro to offset 0
18:14:16.982 [Thread-2] INFO org.apache.iceberg.metrics.LoggingMetricsReporter -- Received metrics report: ScanReport{tableName=rest_catalog.ncentral.customers, snapshotId=8961327026375986033, filter=true, schemaId=0, projectedFieldIds=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27], projectedFieldNames=[city, contact_department, contact_email, contact_ext, contact_firstname, contact_lastname, contact_phone_number, contact_title, country, customer_id, customer_name, external_id, external_id2, is_service_org, is_system, maintenance_window_duratio

In [65]:
import logging

# Suppress all logging
logging.getLogger().setLevel(logging.ERROR)

# If you want to be specific, you can configure the py4j logger as well
logging.getLogger('py4j').setLevel(logging.ERROR)
grouped = joined.groupBy(f"{namespace}.customers.customer_id").count()
grouped.show(10, vertical=True)
print("adsads")


18:14:32.309 [Thread-2] DEBUG org.apache.spark.sql.catalyst.analysis.ResolveReferencesInAggregate -- Resolving 'ncentral.customers.customer_id to customer_id#2267L
18:14:32.310 [Thread-2] DEBUG org.apache.spark.sql.catalyst.analysis.ResolveReferencesInAggregate -- Resolving 'ncentral.customers.customer_id to customer_id#2267L
18:14:32.336 [Thread-2] INFO org.apache.iceberg.spark.source.SparkScanBuilder -- Skipping aggregate pushdown: group by aggregation push down is not supported
18:14:32.340 [Thread-2] INFO org.apache.spark.sql.execution.datasources.v2.V2ScanRelationPushDown -- 
Output: customer_id#2267L
         
18:14:32.341 [Thread-2] INFO org.apache.iceberg.SnapshotScan -- Scanning table rest_catalog.ncentral.customers snapshot 8961327026375986033 created at 2024-06-26T14:54:58.521+00:00 with filter true
18:14:32.344 [Thread-2] INFO org.apache.iceberg.BaseDistributedDataScan -- Planning file tasks locally for table rest_catalog.ncentral.customers
18:14:32.351 [Thread-2] DEBUG org

In [66]:
import logging

# Suppress all logging
logging.getLogger().setLevel(logging.ERROR)

# If you want to be specific, you can configure the py4j logger as well
logging.getLogger('py4j').setLevel(logging.ERROR)
devices = spark.sql("SELECT * FROM ncentral.devices")
devices.show(5)

18:14:57.074 [Thread-2] DEBUG org.apache.spark.sql.execution.SparkSqlParser -- Parsing command: SELECT * FROM ncentral.devices
18:14:57.097 [Thread-2] DEBUG org.apache.iceberg.CachingCatalog -- Evicted ncentral.devices from the table cache (EXPIRED)
18:14:57.100 [Thread-2] DEBUG org.apache.iceberg.shaded.org.apache.hc.client5.http.impl.classic.InternalHttpClient -- ex-0000000050 preparing request execution
18:14:57.100 [Thread-2] DEBUG org.apache.iceberg.shaded.org.apache.hc.client5.http.impl.classic.ProtocolExec -- ex-0000000050 target auth state: UNCHALLENGED
18:14:57.100 [Thread-2] DEBUG org.apache.iceberg.shaded.org.apache.hc.client5.http.impl.classic.ProtocolExec -- ex-0000000050 proxy auth state: UNCHALLENGED
18:14:57.100 [Thread-2] DEBUG org.apache.iceberg.shaded.org.apache.hc.client5.http.impl.classic.ConnectExec -- ex-0000000050 acquiring connection with route {}->http://10.0.0.146:8181
18:14:57.100 [Thread-2] DEBUG org.apache.iceberg.shaded.org.apache.hc.client5.http.impl.cla